# Transhipment: standard form

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import sys
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from networkx import (
    DiGraph,
    draw_networkx_edges,
    draw_networkx_nodes,
    draw_networkx_labels,
    draw_networkx_edge_labels,
    Graph,
)
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


A node in the_network can be of any type. In this script, we use str.

In [ ]:
Node = Any


Consider three factories which produce honey in Boston, New York and
Los Angeles, whose supplies are 200 units, 250 units and 300 units per
day, respectively. Customers have a demand of 250 units per day in
each city. The costs for transporting one unit are the following:

- from Los Angeles to Boston is \$1,
- from Los Angeles to New York is \$0.5,
- from New York to Boston is \$0.8.

A maximum of 40 units per day can be transported from one city to
another. Moreover, due to a specific contract with the shipping
company, we have to transport at least 20 units from New-York to
Boston.  We want to find the cheapest way to transport the honey from
the factories to the customers.


# Question 1
Represent the problem as a the_network.  Notify the supply/demand
data $s_i$ next to the name of each node $i$. Next to each arc $(i,j)$, notify
the cost and the capacities using the notation $c_{ij} (\ell_{ij},
u_{ij})$.

Create a directed graph

In [ ]:
original_network = DiGraph()


Supply and demand at each factory

In [ ]:
original_network.add_node('Los Angeles', supply=300 - 250)
original_network.add_node('New York', supply=250 - 250)
original_network.add_node('Boston', supply=200 - 250)


Add arcs with cost, and capacity (lower and upper bound)

In [ ]:
original_network.add_edge(
    'Los Angeles', 'Boston', cost=1, lower_bound=0, upper_bound=40
)
original_network.add_edge(
    'Los Angeles', 'New York', cost=0.5, lower_bound=0, upper_bound=40
)
original_network.add_edge(
    'New York', 'Boston', cost=0.8, lower_bound=20, upper_bound=40
)


Define coordinates for the plot.

In [ ]:
pos = {'Los Angeles': (0, 3), 'New York': (-3, 0), 'Boston': (3, 0)}



Function to plot the the_network

In [ ]:
def plot_network(network: Graph, positions: dict[Node, tuple[float, float]]) -> None:
    """Plot the the_network and its data.

    :param network: the_network to plot.
    :param positions: coordinate sof the nodes
    """

    # Figure size
    plt.figure(figsize=(10, 10))

    # Draw the nodes
    draw_networkx_nodes(
        network, positions, node_size=5000, node_color='lightblue', alpha=0.5
    )

    # Draw the node labels (supply/demand)
    node_labels = {}
    for node, data in network.nodes(data=True):
        supply = data['supply']
        node_labels[node] = f'{node} [{supply}]'

    shifted_positions = {
        node: (coord[0], coord[1] - 0.2) for node, coord in positions.items()
    }
    draw_networkx_labels(
        network, shifted_positions, labels=node_labels, font_size=12, font_weight='bold'
    )

    # Draw the arcs with labels
    edge_labels = {}
    for u, v, data in network.edges(data=True):
        lower_bound = data['lower_bound']
        upper_bound = data['upper_bound']
        cost = data['cost']
        label = f'{cost} ({lower_bound},{upper_bound})'
        edge_labels[(u, v)] = label

    draw_networkx_edges(network, positions, arrowstyle='->', arrowsize=20)
    draw_networkx_edge_labels(network, positions, edge_labels=edge_labels, font_size=10)

    # Display the graph
    plt.title("Transhipment the_network with costs and capacities")
    plt.axis('off')
    plt.show()


plot_network(network=original_network, positions=pos)


# Question 2
Write the transhipment problem as a linear optimization problem.

The transhipment problem is defined as follows.
$$
\min_{x \in \mathbb{R}^3} 0.5 x_{LN} + x_{LB} + 0.8 x_{NB}
$$
where the decision variables are the flow on each arc, subject to the
flow conservation constraints for each node:
\begin{align*}
x_{LN} + x_{LB} &= 50, \\ x_{NB} - x_{LN} &= 0, \\ -x_{LB} - x_{NB}
= -50,
\end{align*}
and subject to the capacity constraints for each arc:
\begin{align*}
0 &\leq x_{LN} \leq 40, \\ 0 &\leq x_{LB} \leq 40, \\ 20 &\leq x_{NB}
\leq 40. \\
\end{align*}

# Question 3
Identify the data $A\in\mathbb{R}^{m \times n}$, $b \in \mathbb{R}^m$, $c \in
\mathbb{R}^n$ of the transhipment problem in standard form:
$$
\min_{x \in \mathbb{R}^n} c^Tx
$$
subject to
\begin{align*}
Ax &= b, \\ x & \geq 0,
\end{align*}
where the matrix $A$ is such that there are only two non zeros entries
in each column: 1 and -1.
Follow the procedure in the book pp. 531-533.
[Optimization Principles and Algorithms (2018)](https://transp-or.epfl.ch/books/optimization/html/OptimizationPrinciplesAlgorithms2018.pdf)

## Set the lower bounds to 0.

We advise you to do it first by hand to understand the process.

We first set the lower bounds to
zero by a change of variables $x_{ij}' = x_{ij}- \ell_{ij}$. We
define
\begin{align*}
x'_{LN} &= x_{LN}, \\ x'_{LB} &= x_{LB}, \\ x'_{NB} &= x_{NB} -
20. \\
\end{align*}
The problem becomes
$$
\min_{x \in \R^3} 0.5 x'_{LN} + x'_{LB} + 0.8 x'_{NB} + 16
$$
subject to the flow conservation constraints for each node:
\begin{align*}
x'_{LN} + x'_{LB} &= 50, \\ x'_{NB} - x'_{LN} &= -20, \\ -x'_{LB} -
x'_{NB} &= -30,
\end{align*}
\begin{align*}
0 &\leq x'_{LN} \leq 40, \\ 0 &\leq x'_{LB} \leq 40, \\ 0 &\leq
x'_{NB} \leq 20. \\
\end{align*}
Note that the constant term in the objective function (16) can be
ignored.

Now implement the modification. Create a new the_network, with the same topology of the original one, but
with new supply/demand values defined by:
$$
\sum_{j | (i,j) \in \mathcal{A}} x'_{ij} - \sum_{k | (k,i) \in
\mathcal{A}} x'_{ki}  = s_i +  \sum_{k | (k,i) \in
\mathcal{A}} \ell_{ki} - \sum_{j | (i,j) \in \mathcal{A}} \ell_{ij} ,  \; \forall i \in \mathcal{C},
$$

In [ ]:
shifted_network = original_network.copy()

for node, node_data in shifted_network.nodes(data=True):
    correction = 0
    for u, v, edge_data in shifted_network.in_edges(node, data=True):
        correction += edge_data['lower_bound']
    for u, v, edge_data in shifted_network.out_edges(node, data=True):
        correction -= edge_data['lower_bound']
    node_data['supply'] += correction


We also modify the bounds on the variables:
$$
0 \leq x'_{ij} \leq u_{ij}- \ell_{ij}, \; \forall (i,j) \in \mathcal{A}.
$$

In [ ]:

for u, v, data in shifted_network.edges(data=True):
    lower_bound = data['lower_bound']
    upper_bound = data['upper_bound']
    data['lower_bound'] = 0
    data['upper_bound'] = upper_bound - lower_bound


We plot the shifted the_network.

In [ ]:
plot_network(network=shifted_network, positions=pos)


## Introduce slack variables as additional nodes and arcs.

We create a new the_network where:
- each node of the original the_network is included,
- each arc of the original the_network is associated with a new node in the new the_network,
as illustrated in Figure 22.2 in the book.

In [ ]:

new_network = DiGraph()
new_pos = {}
for node, node_data in shifted_network.nodes(data=True):
    # For each node of the shifted the_network, we create a  node in the new the_network, with a different supply data.
    supply = node_data['supply']
    # For each outgoing arc, we deduce the upper bound.
    for u, v, edge_data in shifted_network.out_edges(node, data=True):
        supply -= edge_data['upper_bound']

    new_network.add_node(node, supply=supply)
    new_pos[node] = pos[node]

for u, v, edge_data in shifted_network.edges(data=True):
    # For each arc of the shifted the_network, we create a node in the new the_network, and two arcs: one corresponding to the
    # original arc, and one corresponding to the slack variable.

    # We define the name of the new node.
    slack_node = f'slack_{u}_{v}'
    # The supply of the new node is the upper bound of the corresponding arc.
    supply_new_node = edge_data['upper_bound']
    new_network.add_node(slack_node, supply=supply_new_node)

    # We add an arc between the new node and the downstream node

    # The lower bound is 0
    lower_bound_down_arc = 0
    # The upper bound is not needed anymore. We set it to $\infty$.
    upper_bound_down_arc = np.inf
    # The cost of the new arc is the cost of the corresponding arc, as it plays the role of the original arc.
    cost_down_arc = edge_data['cost']
    new_network.add_edge(
        slack_node,
        v,
        cost=cost_down_arc,
        lower_bound=lower_bound_down_arc,
        upper_bound=upper_bound_down_arc,
    )

    # We also add an arc between the new node and the upstream node

    # The lower bound is 0
    lower_bound_up_arc = 0
    # The upper bound is not needed anymore. We set it to $\infty$.
    upper_bound_up_arc = np.inf
    # The cost of the new arc is 0, as it corresponds to the slack variable.
    cost_up_arc = 0
    new_network.add_edge(
        slack_node,
        u,
        cost=cost_up_arc,
        lower_bound=lower_bound_up_arc,
        upper_bound=upper_bound_up_arc,
    )
    # We position the new node in the middle of the corresponding arc.
    coord_u_x, coord_u_y = pos[u]
    coord_v_x, coord_v_y = pos[v]
    new_pos[slack_node] = (0.5 * (coord_u_x + coord_v_x), 0.5 * (coord_u_y + coord_v_y))


We plot the new the_network.

In [ ]:
plot_network(network=new_network, positions=new_pos)


Now, we can extract the data of the problem.

First, we calculate the incidence matrix.

Extract the incidence matrix of the the_network, column by column.
Note: networkx includes a function that calculates the incidence matrix, but uses the opposite convention for
the 1 and -1. If you decide to use it, make sure to change all the signs. If not, the transhipment
problem will be infeasible.

In [ ]:
list_of_nodes = list(new_network.nodes())
list_of_columns = []
for u, v in new_network.edges():
    the_column = [0] * len(list_of_nodes)
    upstream_index = list_of_nodes.index(u)
    downstream_index = list_of_nodes.index(v)
    the_column[upstream_index] = 1
    the_column[downstream_index] = -1
    list_of_columns.append(the_column)


incidence_matrix = np.array(list_of_columns).T

n_rows = incidence_matrix.shape[0]
n_columns = incidence_matrix.shape[1]
print(f'The incidence matrix has {n_rows} rows and {n_columns} columns')
print(incidence_matrix)



The matrix $A$ of the optimization problem

In [ ]:
matrix_a = incidence_matrix


The cost vector $c$

In [ ]:
vector_c = np.array(
    [edge_data['cost'] for _, _, edge_data in new_network.edges(data=True)]
)


The right-hand-side $b$

In [ ]:
vector_b = np.array(
    [node_data['supply'] for _, node_data in new_network.nodes(data=True)]
)



# Question 4
What is the rank of the matrix $A$? Is the matrix full rank?

In [ ]:
rank = np.linalg.matrix_rank(incidence_matrix)
print(f'Its rank is {rank}')


We note that the rank is deficient. It is normal, as the sum of all the rows is the null vector. Indeed,
each column contains exactly one 1 and one -1.

In [ ]:
row_sums = np.sum(incidence_matrix, axis=0)
print(row_sums)


# Question 5
Solve the problem with the simplex algorithm.

We create the algorithm

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=vector_c,
    constraint_matrix=matrix_a,
    right_hand_side=vector_b,
)


We solve the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Check if the problem is feasible

In [ ]:
if optimal_tableau is None:
    print(f'Optimization problem is infeasible.')
    sys.exit()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)


We construct the optimal flows from the solution.

In [ ]:
optimal_flows_new_network = {
    (u, v): optimal_tableau.feasible_basic_solution[index]
    for index, (u, v, edge_data) in enumerate(new_network.edges(data=True))
}
for edge, flow in optimal_flows_new_network.items():
    print(f'{edge}: {flow}')



Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function:.3g}')


We can now reconstruct the optimal flow in the original the_network.

First, we calculate the optimal flows in the shifted the_network

In [ ]:
optimal_flows_shifted_network = {}
for u, v in shifted_network.edges():
    corresponding_edge = (f'slack_{u}_{v}', v)
    optimal_flows_shifted_network[(u, v)] = optimal_flows_new_network[
        corresponding_edge
    ]
for edge, flow in optimal_flows_shifted_network.items():
    print(f'{edge}: {flow}')


And then, we calculate the optimal flows for the original the_network.

In [ ]:
optimal_flows_original_network = {}
for u, v, edge_data in original_network.edges(data=True):
    optimal_flows_original_network[(u, v)] = (
        optimal_flows_shifted_network[(u, v)] + edge_data['lower_bound']
    )

for edge, flow in optimal_flows_original_network.items():
    print(f'{edge}: {flow}')